<a href="https://colab.research.google.com/github/GarganoNicolas/EVDVai_DataEngTP1/blob/main/EDVai_1_DataEng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
path = 'https://raw.githubusercontent.com/GarganoNicolas/EVDVai_DataEngTP1/main/Utimas_Desvinculaciones.xlsx'

In [ ]:
xls = pd.ExcelFile(path)

sheet_names = xls.sheet_names
print(sheet_names)

['Row data', 'Rangos', 'Managers']


In [ ]:
dfs = {}

for sheet_name in sheet_names:
    dfs[sheet_name] = xls.parse(sheet_name)

data = dfs['Row data']
df_Rangos = dfs['Rangos']
df_Managers = dfs['Managers']

In [ ]:
data.sample(5)

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos
5,15,Analia,Sosa,Compras,2018-10-01,2021-11-30,N4,Grupo A,4,Rango 4,Rango 4,2,NO
25,20,Marcelo,Flores,Ventas,2021-09-06,2022-05-31,N2,Grupo C,6,Rango 2,Rango 2,8,SI
21,9,Nicolas,Martínez,IT,2021-01-04,2021-11-30,N2,Grupo C,6,Rango 3,Rango 3,5,SI
17,25,Carolina,Petersen,Ventas,2021-07-05,2022-02-28,N7,Grupo B,4,Rango 6,Rango 6,8,SI
9,27,Rodrigo,Molinas,Compras,2019-06-03,2022-04-30,N2,Grupo A,4,Rango 2,Rango 2,2,SI


In [ ]:
df_Rangos

,Rango,desde,hasta
0,Rango 1,150000,200000
1,Rango 2,200001,250000
2,Rango 3,250001,300000
3,Rango 4,300001,350000
4,Rango 5,350001,400000
5,Rango 6,400001,500000
6,Rango 7,500001,600000
7,Rango 8,600001,700000
8,Rango 9,700001,800000
9,Rango 10,800001,1000000


In [ ]:
df_Managers

,Id,Nombre,Apellido
0,1,Rolando,Casas
1,2,Marina,Pitana
2,3,José,Feligres
3,4,Miriam,Pisani
4,5,Alejandro,Pitorino
5,6,Sonia,Kamlasky
6,7,Martín,Viglia
7,8,Ricardo,Ronaldo
8,9,Vanina,Quintana
9,10,Daniel,Quiroga


In [ ]:
df_Rangos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rango   10 non-null     object
 1   desde   10 non-null     int64 
 2   hasta   10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 368.0+ bytes


In [ ]:
df_Rangos[['desde', 'hasta']] = df_Rangos[['desde', 'hasta']].astype(str)
df_Rangos['pesitos'] = df_Rangos[['desde', 'hasta']].agg(' - '.join, axis=1)
df_Rangos

,Rango,desde,hasta,pesitos
0,Rango 1,150000,200000,150000 - 200000
1,Rango 2,200001,250000,200001 - 250000
2,Rango 3,250001,300000,250001 - 300000
3,Rango 4,300001,350000,300001 - 350000
4,Rango 5,350001,400000,350001 - 400000
5,Rango 6,400001,500000,400001 - 500000
6,Rango 7,500001,600000,500001 - 600000
7,Rango 8,600001,700000,600001 - 700000
8,Rango 9,700001,800000,700001 - 800000
9,Rango 10,800001,1000000,800001 - 1000000


In [ ]:
# Renombrar columna para hacer un merge
df_Rangos.rename(columns={'Rango': 'Rango Salarial'}, inplace= True)
# Hacer merge
data = data.merge(df_Rangos[['Rango Salarial', 'pesitos']], on='Rango Salarial', how='left')
pesitos = data[['pesitos', 'Rango Salarial']].sort_values('Rango Salarial').drop_duplicates()
pesitos

,pesitos,Rango Salarial
29,150000 - 200000,Rango 1
25,200001 - 250000,Rango 2
2,250001 - 300000,Rango 3
1,300001 - 350000,Rango 4
20,350001 - 400000,Rango 5
10,400001 - 500000,Rango 6


In [ ]:
data.sample(3)

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos
9,27,Rodrigo,Molinas,Compras,2019-06-03,2022-04-30,N2,Grupo A,4,Rango 2,Rango 2,2,SI,200001 - 250000
29,30,Diego,Buzos,Auditoria,2021-09-06,2022-06-30,N1,Grupo C,3,Rango 1,Rango 1,10,SI,150000 - 200000
28,29,Dante,Martini,IT,2021-03-01,2021-12-31,N4,Grupo C,6,Rango 3,Rango 3,5,NO,250001 - 300000


In [ ]:
Rango_salarial = data[(data['Rango Salarial'] != data['RS competencia'])]
Rango_salarial[['Id', 'Rango Salarial', 'RS competencia', 'Area']]

,Id,Rango Salarial,RS competencia,Area
4,13,Rango 4,Rango 3,IT
7,19,Rango 4,Rango 5,Auditoria
18,26,Rango 2,Rango 3,Producción
20,5,Rango 5,Rango 4,Marketing
26,21,Rango 6,Rango 5,Compras


In [ ]:
df_Managers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Id        10 non-null     int64 
 1   Nombre    10 non-null     object
 2   Apellido  10 non-null     object
dtypes: int64(1), object(2)
memory usage: 368.0+ bytes


In [ ]:
# Lo mismo que con Rango, pero ahora con dos columnas
df_Managers.rename(columns={'Id': 'Manager', 'Nombre': 'Nombre del Manager', 'Apellido': 'Apellido del Manager'}, inplace=True)
data = data.merge(df_Managers[['Manager', 'Nombre del Manager', 'Apellido del Manager']], on='Manager', how='left')
data.sample(3)

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos,Nombre del Manager,Apellido del Manager
21,9,Nicolas,Martínez,IT,2021-01-04,2021-11-30,N2,Grupo C,6,Rango 3,Rango 3,5,SI,250001 - 300000,Alejandro,Pitorino
18,26,Fabian,Quiroz,Producción,2018-05-07,2021-09-30,N3,Grupo B,3,Rango 2,Rango 3,6,SI,200001 - 250000,Sonia,Kamlasky
15,18,Pedro,Torres,Marketing,2017-04-03,2019-12-31,N3,Grupo B,4,Rango 3,Rango 3,9,SI,250001 - 300000,Vanina,Quintana


In [ ]:
# Quiero saber si se fue algun manager...
apellido = data[(data['Nombre'] == data['Nombre del Manager'])]
apellido[['Nombre', 'Apellido', 'Nombre del Manager', 'Apellido del Manager']]

,Nombre,Apellido,Nombre del Manager,Apellido del Manager


In [ ]:
def remove_non_numeric(s):
    return re.sub('[^0-9]+', '', s)

# Assuming 'data' is your DataFrame and it's already defined
mean_values = data[['Area', 'Manager', 'Grupo recruitment', 'Tiempo recruitment', 'Rango Salarial', 'Nivel']]
mean_values[['Rango Salarial', 'Nivel']] = mean_values[['Rango Salarial', 'Nivel']].apply(lambda x: x.apply(remove_non_numeric))
mean_values[['Rango Salarial', 'Nivel']] = mean_values[['Rango Salarial', 'Nivel']].apply(pd.to_numeric, errors='coerce')

# List of column names to group by
group_columns = ['Area', 'Manager', 'Grupo recruitment']

# Dictionary to store the results for each group
group_results = {}

for column in group_columns:
    # Grouping and calculating mean values
    mean_values_group = mean_values.groupby(column)[['Tiempo recruitment', 'Rango Salarial', 'Nivel']].mean()
    mean_values_group.reset_index(inplace=True)
    mean_values_group.columns = [column, 'Prom Tiempo recruitment', 'Prom Rango Salarial', 'Prom Nivel']

    # Merging the mean values back to the original DataFrame
    merged_data = mean_values.merge(mean_values_group, on=column, how='left')

    value_counts = data.groupby(column)[column].transform('count')
    merged_data[f'{column}_bajas'] = value_counts

    # Dropping unnecessary columns and duplicates
    merged_data = merged_data.drop(['Tiempo recruitment', 'Rango Salarial', 'Nivel'], axis=1)


    # Storing the result in the dictionary
    if column == 'Manager':
        group_results[column] = merged_data[[column, 'Area', 'Prom Tiempo recruitment', 'Prom Rango Salarial', 'Prom Nivel', f'{column}_bajas']].drop_duplicates().sort_values(column).reset_index(drop=True)
    else:
        group_results[column] = merged_data[[column, 'Prom Tiempo recruitment', 'Prom Rango Salarial', 'Prom Nivel', f'{column}_bajas']].drop_duplicates().sort_values(column).reset_index(drop=True)






# Print the results
for column, result in group_results.items():
    print(f'\n\n promedios por {column}')
    display(result)





 promedios por Area


<ipython-input-325-7d6f51be5d94>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mean_values[['Rango Salarial', 'Nivel']] = mean_values[['Rango Salarial', 'Nivel']].apply(lambda x: x.apply(remove_non_numeric))
<ipython-input-325-7d6f51be5d94>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mean_values[['Rango Salarial', 'Nivel']] = mean_values[['Rango Salarial', 'Nivel']].apply(pd.to_numeric, errors='coerce')


,Area,Prom Tiempo recruitment,Prom Rango Salarial,Prom Nivel,Area_bajas
0,Auditoria,3.5,2.500000,2.500000,2
1,Compras,3.8,3.400000,3.400000,5
2,IT,4.5,3.875000,3.750000,8
3,Legales,4.0,2.000000,2.000000,1
4,Marketing,4.5,4.000000,4.000000,4
5,Producción,4.0,4.000000,4.500000,4
6,Ventas,4.5,4.166667,4.166667,6




 promedios por Manager


,Manager,Area,Prom Tiempo recruitment,Prom Rango Salarial,Prom Nivel,Manager_bajas
0,1,Marketing,4.5,4.000000,4.000000,2
1,2,Compras,3.8,3.400000,3.400000,5
2,3,Legales,4.0,2.000000,2.000000,1
3,4,Producción,4.5,4.000000,4.000000,2
4,5,IT,5.4,4.400000,4.200000,5
5,6,Producción,3.5,4.000000,5.000000,2
6,7,IT,3.0,3.000000,3.000000,3
7,8,Ventas,4.5,4.166667,4.166667,6
8,9,Marketing,4.5,4.000000,4.000000,2
9,10,Auditoria,3.5,2.500000,2.500000,2




 promedios por Grupo recruitment


,Grupo recruitment,Prom Tiempo recruitment,Prom Rango Salarial,Prom Nivel,Grupo recruitment_bajas
0,Grupo A,3.727273,3.636364,3.727273,11
1,Grupo B,3.625000,3.375000,3.750000,8
2,Grupo C,5.181818,4.090909,3.818182,11


In [ ]:
data['Fecha comienzo'] = pd.to_datetime(data['Fecha comienzo'])
data['Fecha fin'] = pd.to_datetime(data['Fecha fin'])
print(
    'Fecha comienzo:',
    ' min ',
    data['Fecha comienzo'].min(),
    ' max ',
    data['Fecha comienzo'].max(),
    '\nFecha fin:',
    ' min ',
    data['Fecha fin'].min(),
    ' max ',
    data['Fecha fin'].max()
    )

Fecha comienzo:  min  2017-01-02 00:00:00  max  2022-01-03 00:00:00 
Fecha fin:  min  2017-11-30 00:00:00  max  2022-07-01 00:00:00


In [ ]:
start_date = data['Fecha comienzo'].min()
end_date = data['Fecha fin'].max()

date_range = pd.date_range(start=start_date, end=end_date, freq='M')
calendario = pd.DataFrame(date_range, columns=['Fecha']).T
calendario

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65
Fecha,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30


In [ ]:
start_date = data['Fecha comienzo'].min()
end_date = data['Fecha fin'].max()

date_range = pd.date_range(start=start_date, end=end_date, freq='M')
calendario = pd.DataFrame(date_range, columns=['Fecha'])
calendario['Fecha'] = calendario['Fecha'].dt.strftime('%Y-%m')
print(calendario['Fecha'].dtype)
calendario.set_index('Fecha', inplace=True)
calendario=calendario.T
calendario

object


Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06


In [ ]:
calendario1 = calendario.copy()
calendario.insert(loc=0, column='Id', value=data['Id'])
calendario

Fecha,Id,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
calendario.set_index('Id', inplace=True)
calendario

Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data1 = data.copy()
data1['Fecha comienzo'] = data1['Fecha comienzo'].dt.strftime('%Y-%m')
data1['Fecha fin'] = data1['Fecha fin'].dt.strftime('%Y-%m')
selected_row = data1.loc[data1['Id'] == 4]

selected_row

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos,Nombre del Manager,Apellido del Manager
0,4,Sebastian,González,Legales,2018-05,2021-05,N2,Grupo A,4,Rango 2,Rango 2,3,SI,200001 - 250000,José,Feligres


In [ ]:

# Iterate over all columns in calendario
for column_name in calendario.columns:
    # Check if the value in 'Fecha comienzo' of selected_row matches the current column name
    value = 1 if selected_row['Fecha comienzo'].iloc[0] == column_name else 0

    # Update the value in the current column of calendario for the row with index 4
    calendario.at[4, column_name] = value

    # Break the iteration if the value is 1
    if value == 1:
        break

# Display the updated calendario DataFrame
calendario.head(3)



Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data1 = data.copy()
data1['Fecha comienzo'] = data1['Fecha comienzo'].dt.strftime('%Y-%m')
data1['Fecha fin'] = data1['Fecha fin'].dt.strftime('%Y-%m')
selected_row = data1.loc[data1['Id'] == 4]

selected_row


# Flag to track the column where the value was 1
column_start = 0

# Iterate over all columns in calendario
for column_name in calendario.columns[column_start:]:
    # Check if the value in 'Fecha comienzo' of selected_row matches the current column name
    value = 1 if selected_row['Fecha comienzo'].iloc[0] == column_name else 0

    # Update the value in the current column of calendario for the row with index 4
    calendario.at[4, column_name] = value

    # Break the iteration if the value is 1
    if value == 1:
        column_start = calendario.columns.get_loc(column_name) + 1
        break

# If the value was 1, iterate from the next column
for column_name in calendario.columns[column_start:]:
    # Check the 'Fecha fin' condition
    value = 0 if selected_row['Fecha fin'].iloc[0] == column_name else 1

    # Update the value in the current column of calendario for the row with index 4
    calendario.at[4, column_name] = value

    # Break the iteration if the value is 0
    if value == 0:
        break

# After the second iteration, check if 'Fecha fin' is the same as the column name and fill with 1
if selected_row['Fecha fin'].iloc[0] == column_name:
    calendario.at[4, column_name] = 1

# Display the updated calendario DataFrame
pd.set_option('display.max_columns', None)
calendario.head(3)






Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Assuming 'data' is your original DataFrame and 'cacalendario' is your target DataFrame
data1 = data.copy()
data1['Fecha comienzo'] = data1['Fecha comienzo'].dt.strftime('%Y-%m')
data1['Fecha fin'] = data1['Fecha fin'].dt.strftime('%Y-%m')

# Iterate over each row in data1
for index, row in data1.iterrows():
    selected_row = row

    calendario_row_index = selected_row['Id']
    column_start = 0

    # Iterate over all columns in cacalendario
    for column_name in calendario.columns[column_start:]:
        # Check if the value in 'Fecha comienzo' of selected_row matches the current column name
        value = 1 if selected_row['Fecha comienzo'] == column_name else 0

        # Update the value in the current column of cacalendario for the row with index cacalendario_row_index
        calendario.at[calendario_row_index, column_name] = value

        # Break the iteration if the value is 1
        if value == 1:
            column_start = calendario.columns.get_loc(column_name) + 1
            break

    # If the value was 1, iterate from the next column
    for column_name in calendario.columns[column_start:]:
        # Check the 'Fecha fin' condition
        value = 0 if selected_row['Fecha fin'] == column_name else 1

        # Update the value in the current column of cacalendario for the row with index cacalendario_row_index
        calendario.at[calendario_row_index, column_name] = value

        # Break the iteration if the value is 0
        if value == 0:
            # Calculate the starting column index for the fill operation
            fill_start_index = calendario.columns.get_loc(column_name) + 1
            # Fill all the next columns with 0
            calendario.loc[calendario_row_index, calendario.columns[fill_start_index:]] = 0
            break


# Convert all columns in cacalendario to integers
calendario = calendario.astype(int)

# Display the updated cacalendario DataFrame
pd.set_option('display.max_columns', None)
calendario


Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0
19,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0


In [ ]:
Trabajadores = calendario.sum().to_frame()
Trabajadores.rename(columns={0: 'n_trabajadores'}, inplace=True)
Trabajadores.T

Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
n_trabajadores,1,1,3,4,5,5,5,5,5,5,5,5,5,6,7,7,9,10,10,10,10,11,11,11,12,13,13,13,13,14,15,16,16,16,16,15,17,17,16,15,15,16,18,18,18,18,17,15,15,15,15,15,14,12,13,13,14,14,12,9,10,8,7,5,3,1


In [ ]:
Trabajadores['relacion_altasBajas'] = Trabajadores['n_trabajadores'].diff()
Trabajadores.T

Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
n_trabajadores,1.0,1.0,3.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,6.0,7.0,7.0,9.0,10.0,10.0,10.0,10.0,11.0,11.0,11.0,12.0,13.0,13.0,13.0,13.0,14.0,15.0,16.0,16.0,16.0,16.0,15.0,17.0,17.0,16.0,15.0,15.0,16.0,18.0,18.0,18.0,18.0,17.0,15.0,15.0,15.0,15.0,15.0,14.0,12.0,13.0,13.0,14.0,14.0,12.0,9.0,10.0,8.0,7.0,5.0,3.0,1.0
relacion_altasBajas,NaN,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,-1.0,2.0,0.0,-1.0,-1.0,0.0,1.0,2.0,0.0,0.0,0.0,-1.0,-2.0,0.0,0.0,0.0,0.0,-1.0,-2.0,1.0,0.0,1.0,0.0,-2.0,-3.0,1.0,-2.0,-1.0,-2.0,-2.0,-2.0


In [ ]:
altas = data['Fecha comienzo'].dt.strftime('%Y-%m')
altas.sort_values()

2     2017-01
3     2017-03
12    2017-03
15    2017-04
20    2017-05
26    2017-11
7     2018-02
11    2018-03
0     2018-05
18    2018-05
19    2018-06
5     2018-10
16    2018-12
8     2019-01
23    2019-02
13    2019-04
9     2019-06
14    2019-07
24    2019-08
10    2020-01
6     2020-01
4     2020-06
22    2020-07
1     2020-07
21    2021-01
28    2021-03
17    2021-07
25    2021-09
29    2021-09
27    2022-01
Name: Fecha comienzo, dtype: object

In [ ]:
bajas = data['Fecha fin'].dt.strftime('%Y-%m')
bajas.sort_values()

20    2017-11
11    2018-12
19    2019-04
15    2019-12
23    2020-03
24    2020-04
3     2020-11
22    2020-12
26    2020-12
7     2021-01
1     2021-03
0     2021-05
16    2021-06
2     2021-06
18    2021-09
5     2021-11
21    2021-11
8     2021-12
12    2021-12
28    2021-12
4     2022-02
17    2022-02
13    2022-03
14    2022-04
9     2022-04
25    2022-05
6     2022-05
10    2022-06
29    2022-06
27    2022-07
Name: Fecha fin, dtype: object

In [ ]:
Trabajadores['n_altas'] = 0
altas_counts = altas.value_counts()
for date, count in altas_counts.items():
    Trabajadores.loc[Trabajadores.index == date, 'n_altas'] = count
Trabajadores['n_bajas'] = 0
bajas_counts = bajas.value_counts()
for date, count in bajas_counts.items():
    Trabajadores.loc[Trabajadores.index == date, 'n_bajas'] = count
Trabajadores.T

Fecha,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06
n_trabajadores,1.0,1.0,3.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,6.0,7.0,7.0,9.0,10.0,10.0,10.0,10.0,11.0,11.0,11.0,12.0,13.0,13.0,13.0,13.0,14.0,15.0,16.0,16.0,16.0,16.0,15.0,17.0,17.0,16.0,15.0,15.0,16.0,18.0,18.0,18.0,18.0,17.0,15.0,15.0,15.0,15.0,15.0,14.0,12.0,13.0,13.0,14.0,14.0,12.0,9.0,10.0,8.0,7.0,5.0,3.0,1.0
relacion_altasBajas,NaN,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,-1.0,2.0,0.0,-1.0,-1.0,0.0,1.0,2.0,0.0,0.0,0.0,-1.0,-2.0,0.0,0.0,0.0,0.0,-1.0,-2.0,1.0,0.0,1.0,0.0,-2.0,-3.0,1.0,-2.0,-1.0,-2.0,-2.0,-2.0
n_altas,1.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
n_bajas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,2.0,3.0,0.0,2.0,1.0,2.0,2.0,2.0


In [ ]:
altas_anual = data['Fecha comienzo'].dt.strftime('%Y').value_counts()
bajas_anual = data['Fecha fin'].dt.strftime('%Y').value_counts()

# Convert each Series to a DataFrame and reset the index
altas_anual_df = altas_anual.to_frame(name='altas').reset_index()
bajas_anual_df = bajas_anual.to_frame(name='bajas').reset_index()

# Rename the index column to 'Año'
altas_anual_df.rename(columns={'index': 'Año'}, inplace=True)
bajas_anual_df.rename(columns={'index': 'Año'}, inplace=True)

# Merge the two DataFrames on 'Año', using a left join to include all years from altas_anual
relacion_altas_bajas_anual = pd.merge(altas_anual_df, bajas_anual_df, on='Año', how='left')

# Calculate the difference between altas and bajas and add it as a new column
relacion_altas_bajas_anual['dif'] = relacion_altas_bajas_anual['altas'] - relacion_altas_bajas_anual['bajas']

# Append a final row with the total counts of altas and bajas
total_altas = altas_anual.sum()
total_bajas = bajas_anual.sum()
total_dif = total_altas - total_bajas
relacion_altas_bajas_anual = relacion_altas_bajas_anual.append({'Año': 'Total', 'altas': total_altas, 'bajas': total_bajas, 'dif': total_dif}, ignore_index=True)

relacion_altas_bajas_anual.sort_values('Año')


<ipython-input-340-41f87ab75f05>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  relacion_altas_bajas_anual = relacion_altas_bajas_anual.append({'Año': 'Total', 'altas': total_altas, 'bajas': total_bajas, 'dif': total_dif}, ignore_index=True)


,Año,altas,bajas,dif
1,2017,6,1,5
0,2018,7,1,6
2,2019,6,2,4
3,2020,5,5,0
4,2021,5,11,-6
5,2022,1,10,-9
6,Total,30,30,0
